# Sverigedemokraternas budgetmotion 2022

https://www.riksdagen.se/sv/dokument-lagar/dokument/motion/nu-ar-det-dags-for-sverige--sverigedemokraternas_H9023938

In [1]:
import re
from io import StringIO
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
multiplier = 1_000

In [3]:
docs = [
    {'utgiftsområde': 1, 'dok_id': 'H9023907'},
    {'utgiftsområde': 2, 'dok_id': 'H9023906'},
    {'utgiftsområde': 3, 'dok_id': 'H9023905'},
    {'utgiftsområde': 4, 'dok_id': 'H9023904'},
    {'utgiftsområde': 5, 'dok_id': 'H9023903'},
    {'utgiftsområde': 6, 'dok_id': 'H9023902'},
    {'utgiftsområde': 7, 'dok_id': 'H9023901'},
    {'utgiftsområde': 8, 'dok_id': 'H9023900'},
    {'utgiftsområde': 9, 'dok_id': 'H9023899'},
    {'utgiftsområde': 10, 'dok_id': 'H9023898'},
    {'utgiftsområde': 11, 'dok_id': 'H9023897'},
    {'utgiftsområde': 12, 'dok_id': 'H9023896'},
    {'utgiftsområde': 13, 'dok_id': 'H9023895'},
    {'utgiftsområde': 14, 'dok_id': 'H9023894'},
    {'utgiftsområde': 15, 'dok_id': 'H9023893'},
    {'utgiftsområde': 16, 'dok_id': 'H9023892'},
    {'utgiftsområde': 17, 'dok_id': 'H9023885'},
    {'utgiftsområde': 18, 'dok_id': 'H9023886'},
    {'utgiftsområde': 20, 'dok_id': 'H9023887'},
    {'utgiftsområde': 21, 'dok_id': 'H9023921'},
    {'utgiftsområde': 22, 'dok_id': 'H9023888'},
    {'utgiftsområde': 23, 'dok_id': 'H9023889'},
    {'utgiftsområde': 24, 'dok_id': 'H9023890'},
    {'utgiftsområde': 25, 'dok_id': 'H9023891'}]

In [4]:
resp = requests.get('https://data.riksdagen.se/dokument/H9023885.html')

In [6]:
def collect_tables(url):
    resp = requests.get(url)
    text = resp.text
    soup = BeautifulSoup(text)
    tables = soup.select('h3+table')
    frames = []

    for table in tables:
        headline = table.findPreviousSibling('h3').text.strip()
        if not re.match('\d+:\d+', headline):
            continue
        frame = pd.read_html(StringIO(str(table)), header=0, thousands=' ')[0]
        frame['Anslag'] = headline
        # Misspelled header
        frame = frame.rename(columns={'Beräknat 2023.1': 'Beräknat 2024'})
        # Other misspelled header
        if 'Förslag 2021' in frame:
            frame = frame.rename(columns={'Beräknat 2023': 'Beräknat 2024',
                                          'Beräknat 2022': 'Beräknat 2023',
                                          'Förslag 2021': 'Förslag 2022'})
        frames.append(frame)
    
    try:
        df = pd.concat(frames)
    except ValueError:
        df = None
    return df

In [7]:
frames = []

for doc in docs:
    url = f'http://data.riksdagen.se/dokument/{doc["dok_id"]}.html'
    frame = collect_tables(url)
    if frame is None:
        print(doc['utgiftsområde'])
        continue
    frame['Utgiftsområde'] = doc['utgiftsområde']
    frames.append(frame)

2
3
5


In [21]:
df = pd.concat(frames)

In [31]:
# Fix error
df['Beräknat 2024'] = df['Beräknat 2024'].fillna(df['Beräknat 2024 4'])
df = df.drop('Beräknat 2024 4', axis=1)

In [32]:
# Special case for utgiftsområde 2
# https://riksdagen.se/sv/dokument-lagar/dokument/motion/utgiftsomrade-2-samhallsekonomi-och_H9023906
df = df.append([{
    'Anslag': '1:11 Finansinspektionen',
    'Förslag 2022': '5000',
    'Beräknat 2023': '5000',
    'Beräknat 2024': '5000',
    'Utgiftsområde': 2,
}])

In [33]:
# Special case for utgiftsområde 3
# https://riksdagen.se/sv/dokument-lagar/dokument/motion/utgiftsomrade-3-skatt-tull-och-exekution_H9023905
df = df.append([
    {
        'Anslag': '1:1 Skatteverket',
        'Förslag 2022': '415000',
        'Beräknat 2023': '350000',
        'Beräknat 2024': '271000',
        'Utgiftsområde': 3
    },
    {
        'Anslag': '1:2 Tullverket',
        'Förslag 2022': '90000',
        'Beräknat 2023': '190000',
        'Beräknat 2024': '290000',
        'Utgiftsområde': 3
    },
    {
        'Anslag': '1:3 Kronofogdemyndigheten',
        'Förslag 2022': '50000',
        'Beräknat 2023': '50000',
        'Beräknat 2024': '50000',
        'Utgiftsområde': 3
    },
    {
        'Anslag': '99:1 Folkräkning',
        'Förslag 2022': '500000',
        'Beräknat 2023': '0',
        'Beräknat 2024': '0',
        'Utgiftsområde': 3
    }])

In [34]:
# Special case for utgiftsområde 5
# https://riksdagen.se/sv/dokument-lagar/dokument/motion/utgiftsomrade-2-samhallsekonomi-och_H9023906
df = df.append([
    {
        'Anslag': '1:3 Nordiskt samarbete',
        'Förslag 2022': '10000',
        'Beräknat 2023': '10000',
        'Beräknat 2024': '10000',
        'Utgiftsområde': 5
    },
    {
        'Anslag': '1:5 Inspektionen för strategiska produkter',
        'Förslag 2022': '1000',
        'Beräknat 2023': '1000',
        'Beräknat 2024': '1000',
        'Utgiftsområde': 5
    },
    {
        'Anslag': '1:9 Svenska institutet',
        'Förslag 2022': '10000',
        'Beräknat 2023': '10000',
        'Beräknat 2024': '10000',
        'Utgiftsområde': 5
    },])

In [35]:
df = df[df['Unnamed: 0'] != 'Avvikelse från regeringen totalt:']

In [36]:
# Hack to sum up too detailed rows
df = df.drop(['Avvikelse från regeringen', 'Unnamed: 0'], axis=1)
df.columns = [x.replace('Förslag ', '').replace('Beräknat ', '') for x in df.columns]
for col in ['2022', '2023', '2024']:
    df[col] = df[col].fillna(0).astype(str)
    df[col] = df[col].str.split('.', expand=True)[0]
    df[col] = df[col].str.replace('±0', '0', regex=False)
    df[col] = df[col].str.replace('\s+', '', regex=True)
    df[col] = df[col].str.replace('−', '-')
    df[col] = df[col].str.replace('–', '-')
    df[col] = df[col].astype(int) * multiplier

df = df.reset_index(drop=True)
_map = df[['Anslag', 'Utgiftsområde']].drop_duplicates().set_index('Anslag')['Utgiftsområde'].to_dict()
df = df.groupby('Anslag', as_index=False).sum().drop('Utgiftsområde', axis=1)
df['Utgiftsområde'] = df.Anslag.map(_map)
df = df.sort_values('Utgiftsområde')

In [37]:
df['Namn'] = df.Anslag.str.split('\d+:\d+', expand=True)[1].str.strip()
df['Anslag'] = df.Anslag.str.extract('^(\d+:\d+)')

In [40]:
df.to_csv('../data/budgetmotion-2022-sd.csv', index=False)